# **installation de la bibliotheque NEO4j**

In [1]:
! pip install neo4j
! pip install sqlite3
! pip install os


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 4.8 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


# Migration de données du modèle relationnel vers la base postgres

In [ ]:
import os
import sqlite3
from neo4j import GraphDatabase

# Fonction pour se connecter à SQLite avec un chemin relatif
def connect_to_sqlite(db_name="infractions.db"):
    db_path = os.path.join(os.path.dirname(__file__), db_name)
    if not os.path.exists(db_path):
        raise FileNotFoundError(f"Le fichier {db_name} est introuvable dans le dossier du script.")
    return sqlite3.connect(db_path)

# Fonction pour se connecter à Neo4j
def connect_to_neo4j():
    uri = input("Entrez l'URI de Neo4j (par défaut: bolt://localhost:7687): ") or "bolt://localhost:7687"
    user = input("Entrez le nom d'utilisateur Neo4j (par défaut: neo4j): ") or "neo4j"
    password = input("Entrez le mot de passe Neo4j: ")
    return GraphDatabase.driver(uri, auth=(user, password))

# Fonction pour insérer un nœud dans Neo4j
def create_node(tx, label, properties):
    query = f"CREATE (n:{label} {{ {', '.join([f'{key}: ${key}' for key in properties])} }})"
    tx.run(query, **properties)

# Fonction pour créer une relation entre deux nœuds
def create_relationship(tx, start_label, start_key, end_label, end_key, rel_type):
    query = f"""
    MATCH (a:{start_label} {{ {start_key[0]}: ${start_key[1]} }}), (b:{end_label} {{ {end_key[0]}: ${end_key[1]} }})
    CREATE (a)-[:{rel_type}]->(b)
    """
    tx.run(query, **{start_key[1]: start_key[2], end_key[1]: end_key[2]})

# Fonction principale de migration
def migrate_data(sqlite_conn, neo4j_driver):
    sqlite_cursor = sqlite_conn.cursor()
    with neo4j_driver.session() as session:

        # Table infractions -> Noeuds (Infraction)
        sqlite_cursor.execute("SELECT code_infr, lib_infr FROM infractions")
        for code_infr, lib_infr in sqlite_cursor.fetchall():
            session.write_transaction(create_node, "Infraction", {"code_infr": code_infr, "lib_infr": lib_infr})

        # Table départements -> Noeuds (Département)
        sqlite_cursor.execute("SELECT code_dep, nom_dep FROM départements")
        for code_dep, nom_dep in sqlite_cursor.fetchall():
            session.write_transaction(create_node, "Departement", {"code_dep": code_dep, "nom_dep": nom_dep})

        # Table force_odre -> Noeuds (ForceOrdre)
        sqlite_cursor.execute("SELECT force_odre FROM force_odre")
        for force_odre, in sqlite_cursor.fetchall():
            session.write_transaction(create_node, "ForceOrdre", {"force_odre": force_odre})

        # Table brigade -> Noeuds (Brigade) avec relations vers Département et ForceOrdre
        sqlite_cursor.execute("SELECT id_brigade, lib_brigade, lib_perimetre, code_dep, force_odre FROM brigade")
        for id_brigade, lib_brigade, lib_perimetre, code_dep, force_odre in sqlite_cursor.fetchall():
            session.write_transaction(create_node, "Brigade", {
                "id_brigade": id_brigade,
                "lib_brigade": lib_brigade,
                "lib_perimetre": lib_perimetre,
                "code_dep": code_dep,
                "force_odre": force_odre
            })

        # Table fait -> Relation entre Brigade et Infraction
        sqlite_cursor.execute("SELECT code_infr, id_brigade, annee, nb_infraction FROM fait")
        for code_infr, id_brigade, annee, nb_infraction in sqlite_cursor.fetchall():
            session.write_transaction(create_relationship,
                                     "Brigade", ("id_brigade", "id_brigade", id_brigade),
                                     "Infraction", ("code_infr", "code_infr", code_infr),
                                     "A_COMMIS")

# Point d'entrée du script
if __name__ == "__main__":
    try:
        # Connexion à SQLite
        sqlite_conn = connect_to_sqlite()
        print("Connexion à SQLite réussie.")

        # Connexion à Neo4j
        neo4j_driver = connect_to_neo4j()
        print("Connexion à Neo4j réussie.")

        # Migration des données
        print("Début de la migration...")
        migrate_data(sqlite_conn, neo4j_driver)
        print("Migration terminée avec succès.")

    except Exception as e:
        print(f"Une erreur est survenue : {e}")

    finally:
        # Fermeture des connexions
        if sqlite_conn:
            sqlite_conn.close()
        if neo4j_driver:
            neo4j_driver.close()
